In [1]:
## add ignore warnings for now, will remove and debug once full algorithm is complete
import warnings
warnings.filterwarnings("ignore")

## import packages/libraries
from time import perf_counter, clock_gettime_ns, CLOCK_REALTIME
import numpy as np
import pandas as pd
from multiprocessing import Pool, cpu_count
from itertools import product
import sys
import sqlite3

## append filepath to allow files to be called from within project folder
sys.path.append('/home/gerard/Desktop/capstone_project/patoms')
sys.path.append('/home/gerard/Desktop/capstone_project')

## call locally created functions
from snapshot_2d_pattern_v8 import patoms2d
from snapshot_3d_pattern_v6 import patoms3d
from pattern_2d_compare_v5 import pattern_compare_2d
from pattern_3d_compare_v4 import pattern_compare_3d
from get_2d_table_names_v0 import table_names_2d
from updating_ref_table_v0 import update_ref_table
from patom_to_table_v0 import patom_to_table_func

In [2]:
# ## create in memory 2d database
# con2d = sqlite3.connect(":memory:")
# cur2d = con2d.cursor()
# cur2d.execute('create table pat_2d_000000(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time);')
# con2d.commit()

In [3]:
# ## read 2d ref database into memory and convert to ref patoms for comparison
# working_ref_patoms = []
# with open('/home/gerard/Desktop/capstone_project/ref2d_v3.db', 'rb') as f:
#     con2dref = sqlite3.connect(":memory:")
#     con2dref.deserialize(f.read())
#     cur2dref = con2dref.cursor()
#     ref_names = [name for (name,) in cur2dref.execute("select name from sqlite_master where type='table';").fetchall()]
#     for i in ref_names:
#         table = cur2dref.execute(f"select * from {i};").fetchall()
#         # patom = [norm_x, norm_y, pattern_centroid_x, pattern_centroid_y, patom_ind, frame_ind, patom_time]
#         table_array = np.array(table)

#         working_ref_patoms.append(table_array)
# ref_patoms_array = np.vstack(working_ref_patoms[:100])

In [ ]:
# val = 0
# while val <= 3:
#     rand_array = np.random.random((720, 1280))
#     print('rows', rand_array.shape[0])
#     print('cols', rand_array.shape[1])
#     frame_patoms = patoms2d(rand_array)
#     num_patoms = len(frame_patoms)
#     val += 1

In [21]:
import numpy as np

# Example 2D array
array = rand_array = np.random.random((720, 1280))
print(array)

# Threshold for adjacency
threshold = 0.0005

# Step 1: Compute differences along rows and columns
row_diff = np.diff(array, axis=1)  # Differences along rows
col_diff = np.diff(array, axis=0)  # Differences along columns
print(row_diff)
print(col_diff)

# Step 2: Identify where differences exceed the threshold
row_split_mask = row_diff > threshold  # Mask for row splits
col_split_mask = col_diff > threshold  # Mask for column splits
print('rm',col_split_mask)
print('cm',row_split_mask)

# Step 3: Create cumulative group labels for rows and columns
row_groups = np.cumsum(np.pad(row_split_mask, ((0, 0), (1, 0)), constant_values=True), axis=1)
col_groups = np.cumsum(np.pad(col_split_mask, ((1, 0), (0, 0)), constant_values=True), axis=0)
print('cumsum pad rg',row_groups)
print('cumsum pad cg',col_groups)

# Step 4: Combine row and column group labels into unique group labels
combined_groups = row_groups + col_groups * (array.shape[1] + 1)
print('cg',combined_groups)

# Step 5: Extract unique group labels and their indices
unique_groups, group_indices = np.unique(combined_groups, return_inverse=True)
print('ug',unique_groups)
print('gi',group_indices)

# Step 6: Reshape group indices to match the array shape
group_indices = group_indices.reshape(array.shape)
print('rgi', group_indices)

# print([group for group in unique_groups])
# print(array[group_indices])

# Step 7: Group values and indices
grouped_values = [array[group_indices == group] for group in unique_groups]
grouped_indices = [np.argwhere(group_indices == group) for group in unique_groups]

# Output the results
print("Grouped values:")
for group in grouped_values:
    print(group)

print("\nGrouped indices:")
for group in grouped_indices:
    print(group)

[[0.70292061 0.84137195 0.80571106 ... 0.20207314 0.70965429 0.39161769]
 [0.81281006 0.82912171 0.67524427 ... 0.329818   0.00242204 0.94531556]
 [0.84424471 0.61305254 0.11676116 ... 0.16316809 0.70751047 0.98778116]
 ...
 [0.81521086 0.37209305 0.41796672 ... 0.8843639  0.82294463 0.60248757]
 [0.51296875 0.40829331 0.55863676 ... 0.98412488 0.04188998 0.48514741]
 [0.19817096 0.24579471 0.25566852 ... 0.635382   0.17249728 0.79567711]]
[[ 0.13845135 -0.03566089 -0.23525101 ... -0.34874087  0.50758115
  -0.3180366 ]
 [ 0.01631166 -0.15387744  0.1864194  ...  0.27862605 -0.32739595
   0.94289352]
 [-0.23119217 -0.49629137 -0.09566466 ... -0.01162161  0.54434238
   0.28027069]
 ...
 [-0.44311781  0.04587366  0.41951222 ...  0.01511048 -0.06141927
  -0.22045706]
 [-0.10467544  0.15034345  0.02340192 ...  0.71385244 -0.94223489
   0.44325742]
 [ 0.04762375  0.00987381  0.23947825 ... -0.16861438 -0.46288472
   0.62317983]]
[[ 1.09889449e-01 -1.22502416e-02 -1.30466789e-01 ...  1.2774485

KeyboardInterrupt: 